In [4]:
import json
import collections
import operator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn import metrics
from sklearn.cluster import KMeans

In [10]:
users = []
with open('user.json') as json_file:
    for line in json_file:
        users.append(json.loads(line))

In [13]:
user_df = pd.DataFrame.from_dict(users)

In [14]:
user_df

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,3.80,5174,284,5174,5175,78,299,1435,7829,7397,...,16856,"[2014, 2016, 2013, 2011, 2012, 2015, 2010, 2017]",209,"[M19NwFwAXKRZzt8koF11hQ, QRcMZ8pJJBBZaKubHOoMD...",16605,Cin,272,17019,lsSiIjAKVl-QRxKjRErBeg,2010-07-13
1,3.94,1556,211,1556,1285,101,134,1295,162,2134,...,40110,"[2014, 2017, 2011, 2012, 2015, 2009, 2013, 200...",835,"[eoSSJzdprj3jxXyi94vDXg, QF0urZa-0bxga17ZeY-9l...",10882,Andrea,2559,83681,om5ZiponkpRqUNa3pVPiRg,2006-01-18
2,4.72,15,1,15,5,0,1,11,8,20,...,55,[],17,"[Oa84FFGBw1axX8O6uDkmqg, SRcWERSl4rhm-Bz9zN_J8...",4,Gabe,277,45,-lGwMGHMC_XihFJNKCJNRg,2014-10-31
3,3.76,9,0,9,1,0,1,4,1,11,...,4,"[2016, 2017]",11,"[96DJovjKAtExnyBZRfniPQ, Djczq-bRn761U4FW77NUt...",4,Leah,436,15,D-ydMTPGWXTVm4_jjp0k9g,2013-04-01
4,4.23,276,0,276,59,0,8,51,169,386,...,6006,"[2017, 2016]",49,"[iN0A6QIrEFYoSGHFaknh8Q, B2HDoWNIzLlon0IhS1cmD...",360,Juan,921,9152,PcvbBOCOcs6_suRDH7TSTg,2012-08-16
5,5.00,0,0,0,0,0,0,0,0,0,...,1,[],1,"[HanGVaZP_0IwhQdc0F5paQ, _v1_G2Fs56K8JTAzzItAM...",2,Charmaine,22,1,uXjR2GhCAYhqxVr21aC4vQ,2011-10-14
6,4.40,9,0,9,8,0,2,1,1,4,...,0,"[2017, 2016]",17,"[ZWD8UH1T7QXQr0Eq-mcWYg, ObSXluQeEYiOP9l9muFlG...",0,Victoria,91,0,N2arP_u4sMGLgOZhA6ZFoQ,2013-12-20
7,4.06,4,0,4,5,0,5,4,5,5,...,1,"[2016, 2015]",5,"[Puvuej6lzJ1JOEmtjG7V_Q, fq7CL1myWPYeH0d4bKtsI...",3,Erica,68,5,YHJIMK_zVH_VY6HCY6bYvg,2012-06-08
8,3.63,4,0,4,1,0,3,1,0,2,...,2,[],3,"[b8NTnxrhHCs6IPAKrzYL9w, vaNvEgtznFwSPDNkbM7hj...",11,Whitney,184,141,g3V76Ja0XgWS1rqx0gxL_A,2007-09-11
9,4.19,1,0,1,0,0,0,2,0,1,...,2,[],5,"[lM_HnyWHrpBsrgts3VnkxA, ynYGa-R1ZJf-tJOVlQeML...",6,Paul,48,0,VTL4Dep5p7XrrOtgnXdEIQ,2010-04-25


In [15]:
user_df.columns

Index([u'average_stars', u'compliment_cool', u'compliment_cute',
       u'compliment_funny', u'compliment_hot', u'compliment_list',
       u'compliment_more', u'compliment_note', u'compliment_photos',
       u'compliment_plain', u'compliment_profile', u'compliment_writer',
       u'cool', u'elite', u'fans', u'friends', u'funny', u'name',
       u'review_count', u'useful', u'user_id', u'yelping_since'],
      dtype='object')

In [19]:
new_columns = ['average_stars', 'compliment_cool', 'compliment_cute', 'compliment_funny', \
               'compliment_hot', 'compliment_list', 'compliment_more', 'compliment_note', 'compliment_photos', \
               'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool', 'fans', \
               'funny', 'review_count', 'yelping_since']

In [20]:
new_user_df = user_df[new_columns]

In [24]:
def transsformCreatedTime(df, attribute):
    current_time = pd.to_datetime('2017-12-01')
    df[attribute] = df[attribute].map(lambda x : current_time - pd.to_datetime(x))\
                                 .map(lambda x : x / np.timedelta64(1, 's'))\
                                 .map(lambda x : float(x) / (60 * 60 * 24))
    return df[attribute]

In [25]:
new_user_df.yelping_since = transsformCreatedTime(new_user_df, 'yelping_since')

/Users/chuanlong/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/chuanlong/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [26]:
new_user_df

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,cool,fans,funny,review_count,yelping_since
0,3.80,5174,284,5174,5175,78,299,1435,7829,7397,569,1834,16856,209,16605,272,2698.0
1,3.94,1556,211,1556,1285,101,134,1295,162,2134,74,402,40110,835,10882,2559,4335.0
2,4.72,15,1,15,5,0,1,11,8,20,0,1,55,17,4,277,1127.0
3,3.76,9,0,9,1,0,1,4,1,11,0,3,4,11,4,436,1705.0
4,4.23,276,0,276,59,0,8,51,169,386,3,29,6006,49,360,921,1933.0
5,5.00,0,0,0,0,0,0,0,0,0,0,1,1,1,2,22,2240.0
6,4.40,9,0,9,8,0,2,1,1,4,0,1,0,17,0,91,1442.0
7,4.06,4,0,4,5,0,5,4,5,5,1,22,1,5,3,68,2002.0
8,3.63,4,0,4,1,0,3,1,0,2,0,0,2,3,11,184,3734.0
9,4.19,1,0,1,0,0,0,2,0,1,0,3,2,5,6,48,2777.0


In [28]:
user_kmeans = KMeans(n_clusters=5).fit(new_user_df)

In [29]:
user_kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [30]:
user_kmeans.labels_

array([1, 1, 4, ..., 0, 4, 2], dtype=int32)

In [32]:
new_user_df['tag'] = pd.Series(user_kmeans.labels_, index = new_user_df.index) 

/Users/chuanlong/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [33]:
new_user_df

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,cool,fans,funny,review_count,yelping_since,tag
0,3.80,5174,284,5174,5175,78,299,1435,7829,7397,569,1834,16856,209,16605,272,2698.0,1
1,3.94,1556,211,1556,1285,101,134,1295,162,2134,74,402,40110,835,10882,2559,4335.0,1
2,4.72,15,1,15,5,0,1,11,8,20,0,1,55,17,4,277,1127.0,4
3,3.76,9,0,9,1,0,1,4,1,11,0,3,4,11,4,436,1705.0,0
4,4.23,276,0,276,59,0,8,51,169,386,3,29,6006,49,360,921,1933.0,0
5,5.00,0,0,0,0,0,0,0,0,0,0,1,1,1,2,22,2240.0,0
6,4.40,9,0,9,8,0,2,1,1,4,0,1,0,17,0,91,1442.0,0
7,4.06,4,0,4,5,0,5,4,5,5,1,22,1,5,3,68,2002.0,0
8,3.63,4,0,4,1,0,3,1,0,2,0,0,2,3,11,184,3734.0,2
9,4.19,1,0,1,0,0,0,2,0,1,0,3,2,5,6,48,2777.0,2


In [34]:
user_df_id = user_df[['user_id']]

In [36]:
user_df_id['tag'] = pd.Series(user_kmeans.labels_, index = user_df_id.index)

/Users/chuanlong/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
user_df_id

,user_id,tag
0,lsSiIjAKVl-QRxKjRErBeg,1
1,om5ZiponkpRqUNa3pVPiRg,1
2,-lGwMGHMC_XihFJNKCJNRg,4
3,D-ydMTPGWXTVm4_jjp0k9g,0
4,PcvbBOCOcs6_suRDH7TSTg,0
5,uXjR2GhCAYhqxVr21aC4vQ,0
6,N2arP_u4sMGLgOZhA6ZFoQ,0
7,YHJIMK_zVH_VY6HCY6bYvg,0
8,g3V76Ja0XgWS1rqx0gxL_A,2
9,VTL4Dep5p7XrrOtgnXdEIQ,2


In [55]:
dict_userid_tag = collections.defaultdict(int)

In [60]:
userid_tag = user_df_id.values.tolist()

In [62]:
for row in userid_tag:
    dict_userid_tag[row[0]] = row[1]

In [64]:
userid_friends = user_df[['user_id', 'friends']]

In [66]:
userid_friends_list = userid_friends.values.tolist()

In [67]:
new_friends = []
for row in userid_friends_list:
    friends = []
    tag = dict_userid_tag[row[0]]
    for f in row[1]:
        if dict_userid_tag[f] == tag:
            friends.append(f)
    new_friends.append(friends)

In [70]:
user_df_id['friends'] = pd.Series(new_friends, index = user_df_id.index)

/Users/chuanlong/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [71]:
user_df_id

,user_id,tag,friends
0,lsSiIjAKVl-QRxKjRErBeg,1,"[W81-CPVrM9c6F8XiNuEUvA, Wc5L6iuvSNF5WGBlqIO8n..."
1,om5ZiponkpRqUNa3pVPiRg,1,"[jVYzrVblDFSuL3GHtt8ZSA, Lzd7FTvMOvfg9WmR4MoNx..."
2,-lGwMGHMC_XihFJNKCJNRg,4,"[fyg8Dw7BQ8X3Wria7-tHcQ, c_tmM9PQnMDKPhO0-p4kI..."
3,D-ydMTPGWXTVm4_jjp0k9g,0,"[96DJovjKAtExnyBZRfniPQ, dvOmaLaQwJqydq2bM2yAo..."
4,PcvbBOCOcs6_suRDH7TSTg,0,"[B2HDoWNIzLlon0IhS1cmDw, sEYD5YG0fGgvLXAZ5RDuZ..."
5,uXjR2GhCAYhqxVr21aC4vQ,0,"[HanGVaZP_0IwhQdc0F5paQ, _v1_G2Fs56K8JTAzzItAM..."
6,N2arP_u4sMGLgOZhA6ZFoQ,0,"[ZWD8UH1T7QXQr0Eq-mcWYg, RRP7KS85TkbvpB5sF4ANs..."
7,YHJIMK_zVH_VY6HCY6bYvg,0,"[Puvuej6lzJ1JOEmtjG7V_Q, fq7CL1myWPYeH0d4bKtsI..."
8,g3V76Ja0XgWS1rqx0gxL_A,2,"[vaNvEgtznFwSPDNkbM7hjA, tYPh3vjZsi5xMEA561mGp..."
9,VTL4Dep5p7XrrOtgnXdEIQ,2,"[ReZiDOgradY-YmT3nDhKcg, uNOO5YKS6HNQhMbv4C1WQ..."


In [74]:
user_df_id.to_json(path_or_buf = "userid_tag_friends.json", orient = 'records', lines = True)